In [47]:
import pandas as pd
import numpy as np
import time
import os
import zipfile
import pandas_gbq
import glob
from google.cloud import bigquery
import re

In [48]:
client = bigquery.Client()

In [50]:
query = """ 
        SELECT ANO as ano, MES as mes, TIPO_PENSAO as tipo_pensao,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2020_abr_militares_pensionistas_cadastro` 
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_PENSAO as tipo_pensao,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2021_abr_militares_pensionistas_cadastro` 
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_PENSAO as tipo_pensao,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2022_abr_militares_pensionistas_cadastro` 
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_PENSAO as tipo_pensao,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2023_abr_militares_pensionistas_cadastro` 
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_PENSAO as tipo_pensao,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2024_abr_militares_pensionistas_cadastro` 
        """  

In [51]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,mes,tipo_pensao,total
0,2024,4,Não informado,1
1,2024,4,Cônjuge / Viúva (o),1
2,2024,4,Cônjuge / Viúva (o),1
3,2024,4,Cônjuge / Viúva (o),1
4,2024,4,Cônjuge / Viúva (o),1
...,...,...,...,...
1162588,2020,4,Filha,1
1162589,2020,4,Filha,1
1162590,2020,4,Filha,1
1162591,2020,4,Filha,1


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162593 entries, 0 to 1162592
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   ano          1162593 non-null  Int64 
 1   mes          1162593 non-null  Int64 
 2   tipo_pensao  1162593 non-null  object
 3   total        1162593 non-null  Int64 
dtypes: Int64(3), object(1)
memory usage: 38.8+ MB


In [53]:
df['tipo_pensao'].unique()

array(['Não informado', 'Cônjuge / Viúva (o)', 'Companheiro (a)',
       'Pessoa desquitada, separada judicialmente, divorciada do instituidor ou ex-convivente',
       'Filha', 'Filho', 'Mãe', 'Pai', 'Neto (a)', 'Irmão (ã)',
       'Menor sob guarda ou tutela',
       'Pessoa designada (Beneficiário instituído)',
       'Ex-combatente (o próprio)',
       'Outros (Pessoas sem Vínculo Militar)',
       'Filho(a) adotivo ou Enteado(a)'], dtype=object)

In [54]:
pensoes = {
    "Filho" : "Filhos, netos, enteados ou menores em tutela",
    "Filha" : "Filhos, netos, enteados ou menores em tutela",
    "Menor sob guarda ou tutela" : "Filhos, netos, enteados ou menores em tutela",
    "Neto (a)" : "Filhos, netos, enteados ou menores em tutela",
    "Filho(a) adotivo ou Enteado(a)" : "Filhos, netos, enteados ou menores em tutela",
    "Cônjuge / Viúva (o)" : "Cônjuge ou ex-cônjuge",
    "Companheiro (a)" : "Cônjuge ou ex-cônjuge",
    "Pessoa desquitada, separada judicialmente, divorciada do instituidor ou ex-convivente" : "Cônjuge ou ex-cônjuge",
    "Mãe":"Pais",
    "Pai":"Pais",
    "Irmão (ã)":"Irmão (ã)",
    "Ex-combatente (o próprio)":"Ex-combatente (o próprio)",
    "Pessoa designada (Beneficiário instituído)":"Pessoa designada (Beneficiário instituído)",
    "Não informado" : "Não informado",
    "Outros (Pessoas sem Vínculo Militar)" : "Outros (Pessoas sem Vínculo Militar)"
    }


In [55]:
def categorizando(x):
    if x in pensoes:
        return pensoes[x]


In [56]:
df['tipo_pensao_agrupado'] = df['tipo_pensao'].apply(categorizando)

In [59]:
df= df[['ano', 'mes', 'tipo_pensao','tipo_pensao_agrupado', 'total']]
df

,ano,mes,tipo_pensao,tipo_pensao_agrupado,total
0,2024,4,Não informado,Não informado,1
1,2024,4,Cônjuge / Viúva (o),Cônjuge ou ex-cônjuge,1
2,2024,4,Cônjuge / Viúva (o),Cônjuge ou ex-cônjuge,1
3,2024,4,Cônjuge / Viúva (o),Cônjuge ou ex-cônjuge,1
4,2024,4,Cônjuge / Viúva (o),Cônjuge ou ex-cônjuge,1
...,...,...,...,...,...
1162588,2020,4,Filha,"Filhos, netos, enteados ou menores em tutela",1
1162589,2020,4,Filha,"Filhos, netos, enteados ou menores em tutela",1
1162590,2020,4,Filha,"Filhos, netos, enteados ou menores em tutela",1
1162591,2020,4,Filha,"Filhos, netos, enteados ou menores em tutela",1


In [70]:
df['tipo_pensao_agrupado'].unique()

array(['Não informado', 'Cônjuge ou ex-cônjuge',
       'Filhos, netos, enteados ou menores em tutela', 'Pais',
       'Irmão (ã)', 'Pessoa designada (Beneficiário instituído)',
       'Ex-combatente (o próprio)',
       'Outros (Pessoas sem Vínculo Militar)'], dtype=object)

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162593 entries, 0 to 1162592
Data columns (total 5 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   ano                   1162593 non-null  Int64 
 1   mes                   1162593 non-null  Int64 
 2   tipo_pensao           1162593 non-null  object
 3   tipo_pensao_agrupado  1162593 non-null  object
 4   total                 1162593 non-null  Int64 
dtypes: Int64(3), object(2)
memory usage: 47.7+ MB


In [62]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [64]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('mes','INTEGER',description='Mês de referência da observação'),        
        bigquery.SchemaField('tipo_pensao','STRING',description='Tipo de pensão'),
        bigquery.SchemaField('tipo_pensao_agrupado','STRING',description='Categorização das pensões'),
        bigquery.SchemaField('total','INTEGER',description='Quantidade total da observação')
        ]

In [67]:
table_ref = dataset_ref.table('MILITARES_pensionistas_total') 
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 


LoadJob<project=repositoriodedadosgpsp, location=US, id=cca767c1-083a-4141-8a3f-f27423760375>